# Gate Set Tomography

In [46]:
import numpy as np
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiment.library.gateset_tomography import GateSetTomography
from qiskit_experiment.library.gateset_tomography.gatesetbasis import GateSetBasis,default_gateset_basis, gram_matrix_rank

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis
from qiskit.extensions import XGate, YGate, HGate, U2Gate, IGate, SXGate, RZGate
from qiskit.quantum_info import Operator, Choi, PTM, DensityMatrix
from qiskit.quantum_info import DensityMatrix

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## Overview


Gate-Set Tomography is used to perform $\bf{full}$ $\bf{characterization}$ of quantum processes using measurement data we obtain after measuring specific set of quantum circuits.

Unlike the quantum process tomography, gate set tomography $\bf{deals}$ $\bf{with}$the state preparation and measurements $\bf{(SPAM)}$ errors self-consistently by including the gates used for both initializing and measuring the qubits in the gate set which is reconstructed from processing measurement data.

To do this, GST performs full characterization of a set of gates: $(G_0, ..., G_M)$, the native state: $|\rho\rangle$ and the native measurement: $|E\rangle$, where:

$|\rho\rangle$ and $|E\rangle$ are the native preparation state and measurement. The $|\cdot \rangle$ here indicates they are displayed in a superstate form in the Pauli strings basis- also called the PTM representation. 

$G_0, .. ,G_k$ should include the gates we wish to characterize, and additional gates such that 
from set of gates, $[G_i]_{i=0}^{i=M}$, we are able to construct the SPAM gates, $[F_i]_{i=0}^{i=d-1}$, which when applied on the native state and measurement give:

- An informationally complete set of initial states: $|\rho_i\rangle= R_{F_i}|\rho\rangle$, $i=1,2,..,d$

and

- An informationally complete set of measurements: $|E_i\rangle= R_{F_i}|E\rangle$, $i=1,2,..,d$.

** d is the superspace dim.= $2^{2*qubits}$ and R with subscript ${F_i}$ or ${G_k}$ is the PTM representation of $F_i$ and $G_k$ respictively.

Each $F_i$ is constructed from the gates in the gate set. For example, we can have
$F_1=G_5$ and $F_2=G_1o\ G_3$ ($G3$ followed by $G1$).

To be able to characterize the full data, GST uses the following $\bf{three\ sets\ of\ circuits}$: 

1. Circuits that measure elements of the form: $\langle \rho|R_{F_i} R_{G_k} R_{F_j}|E\rangle$

![alt text](GST_circuit_rho_F_G_F_E_element1.png "Title")

2. Circuits that measure elements of the form: $\langle\rho|R_{F_i}R_{F_j}|E\rangle$ (the Gram Matrix -g- Elements)

![alt text](gram_matrix_elements.png "Title")

2. Circuits that measure elements of the form: $\langle\rho|R_{F_i}|E\rangle$ 

![alt text](GST_circuit_rho_F_E_element.png "Title")

#### Analyzing the experimental data

- The above experimental data, is enough to fully characterize the gate set. To do this, we may use two fitters:

$\bf{1)\ Linear\ inversion}$ (the idea is similar to linear inversion in QPT..). But, the final results are true up to some gauge, which is found using scipy optimization method so the final results with the optimized gauge are close as much as possible to some target set. In qiskit GST code, the target is chosen to be the ideal set. 
$\bf{Throwback:}$ This method does not gaurantee the results are TP and CP.

$\bf{2)\ Maximum\ likelihood\ estimation:}$ We find the most likely gate set that gives the true probabilities $p_{ijk}$ based on experimental data $m_{ijk}$ under the physical constraints including CPTP constraints. Here the default is to use the solution obtained by linear inversion after the gauge optimization as a starting point for the optimization problem (the user can choose not to and provide his/her own starting point). $\bf{Advantage:}$ The results satisfy the physical constraints. The problem statement this method attempts to solves is:


![alt text](MLE_problem2.PNG "Title")

where $p_{ijk} = \langle E|R_{F_{i}}R_{G_{k}}R_{F_{j}}|\rho\rangle$, and $|E\rangle$, $R_{F_{i}},\ R_{G_{k}},\ R_{F_{j}},$ and $|\rho\rangle$ are parametrized using a set of parameters $\vec{t}$ that the optimization algorithm should find such that the above statement is satisfied.

# GST Experiment

To run GST experiment, we need to provide the following parameters:

1) qubits: A list of physical qubits GST is performed on.

2) gateset: The gateset, GST experiment characterizes as explained above. More details are in the following section. 

3) additional_gates: List of gates to be added to the gateset to be characterized.

4) only_basis_gates: A boolean variable that indicates whether the gateset argument got a full gatesebasis as a GateSetBasis instance or the 'default' option, or the user provided only basis gates from which the algorithm needs to construct SPAM gates and create the GateSetBasis object.

### GateSetBasis Class:

The gateset we pass in to the GST experiment can be any of the following:
    
a) GateSetBasis instance (A class defined in qiskit_experiment.library.gateset_tomography.gatesetbasis), that is constructed after providing the gateset basis and the SPAM gates. It includes attributes related to the gates in the gate set and SPAM gates. In addition, useful functions that are applied on the gates are included. 

In [31]:
num_qubits=1

# Define the basis gates as a dictionary, where the keys are the names of the gates, and the
#values are the corresponding gates of type: FunctionType (lambda), Gate.
gates_1 = {
    'Id': lambda circ, qubit: None,
    'X_Rot_90': lambda circ, qubit: circ.append(U2Gate(-np.pi / 2, np.pi / 2), [qubit]),
    'Y_Rot_90': lambda circ, qubit: circ.append(U2Gate(0, 0), [qubit])
    }

# Define the SPAM gates which are constructed from the basis gates and give an informationally
#complete set of initial states and measurements when applied on rho=|00><00| and E=|00><00|. 
#The spam dictionary should be of the following form:

spam_1 = {
    'F0': ('Id',),
    'F1': ('X_Rot_90',),
    'F2': ('Y_Rot_90',),
    'F3': ('X_Rot_90', 'X_Rot_90')
}

#Create the GateSetBasis instance
gateset_1=GateSetBasis(name='GST gateset #1', gates = gates_1, spam = spam_1, num_qubits=1)

# The default of only_basis_gates is False, so we don't need to pass it in in this case.
only_basis_gates_1 = False

- Note: For the sake of obtaining good tomography results, it is important to check before running the experiment that the gram matrix has a full rank and not singular or ill-conditioned, so it can be 
inverted appropriately. In addition, the gate set should be picked so the gram matrix singular values are
as large as possible. to display the gram matrix singular values and rank, we can use the gram_matrix_rank function:

In [36]:
gram_rank, singular_values = gram_matrix_rank(num_qubits = 1, spam_gates_labels = list(spam_1.values()) ,basis_gates = gates_1)
print('Gram matrix rank:', gram_rank)
print('Gram matrix singular values:', singular_values)

Gram matrix rank: 4
Gram matrix singular values: [1.78077641 1.         0.5        0.28077641]


b)  A string = 'default' or None, for which the algorithm uses the built-in default gate sets. The default is only available for single qubit or two qubits, and both are built from the backends basis gates.<br><br>
     
$\bf{For \ the \ single-qubit\ case:}$ <br><br>
The gateset basis of the default is: <br>
$\;\;\;$ $G$ = {'I', 'SX','RZ_pi/2', 'RZ_-pi/2'} <br><br>
and the SPAM gates are: <br>
$\;\;\;$ = {<br>
$\;\;\;$    'F0': ('Id',),<br>
$\;\;\;$    'F1': ('SX',),<br>
$\;\;\;$    'F2': ('RZ_pi/2','SX','RZ_-pi/2',),<br>
$\;\;\;$    'F3': ('SX', 'SX')<br>
$\;\;\;$}
<br><br>
Where I is the identity gate IGate(), SX is SXGate(), RZ_pi/2 is RZGate($\theta=\pi/2$) and RZ_-pi/2 is RZGate($\theta=-\pi/2$).<br>
<br><br>
$\bf{For\ the\ two-qubits\ case:}$<br><br>
$\;\;\;$ $G$={'I I','X I', 'I X', 'RZ_pi/3 I', 'I RZ_pi/3', 'I SX', 'SX I', 'CX' )}<br><br>
And the SPAM gates are:<br>
$\;\;\;${<br>
$\;\;\;$    'F0': ('I I',),<br>
$\;\;\;$    'F1': ('X I',),<br>
$\;\;\;$    'F2': ('I X',),<br>
$\;\;\;$    'F3': ('X I', 'I X',),<br>
$\;\;\;$    'F4': ('I X', 'SX I', 'CX',),<br>
$\;\;\;$    'F5': ('I SX', 'I RZ_pi/3', 'I SX',),<br>
$\;\;\;$    'F6': ('SX I', 'RZ_pi/3 I', 'SX I',),<br>
$\;\;\;$    'F7': ('X I', 'I SX', 'I RZ_pi/3', 'I SX'),<br>
$\;\;\;$    'F8': ('I X', 'I SX', 'CX', 'I SX'),<br>
$\;\;\;$    'F9': ('I X', 'SX I', 'RZ I', 'SX I'),<br>
$\;\;\;$    'F10': ('RZ_pi/3 I', 'RZ_pi/3 I', 'RZ_pi/3 I', 'SX I'),<br>
$\;\;\;$    'F11': ('I RZ_pi/3', 'I RZ_pi/3', 'I RZ_pi/3', 'I SX'),<br>
$\;\;\;$    'F12': ('I SX', 'SX I', 'CX', 'I SX'),<br>
$\;\;\;$    'F13': ('X I', 'I RZ_pi/3', 'I RZ_pi/3', 'I RZ_pi/3', 'I SX'),<br>
$\;\;\;$    'F14': ('I SX', 'I RZ_pi/3', 'SX I', 'CX', 'I SX'),<br>
$\;\;\;$    'F15': ('RZ_pi/3 I', 'RZ_pi/3 I', 'RZ_pi/3 I', 'CX', 'I SX', 'CX')<br>
$\;\;\;$},<br>
where here, 'A B' stands for applying a gate B on the first qubit, and gate A on the second qubit. 'RZ_pi/3' is the $RZGate(\theta=\pi/3)$.


In [37]:
gateset_2 = 'default'

# Alternatively,
# gateset_2 = None
# or we can pass the default as a GateSetBasis:
# gateset_2 = default_gateset_basis(num_qubits)

only_basis_gates_2 = False

c) Only a set of basis gates the gate set is composed of. In this case, the boolean variable
'only_basis_gates' should take the True value. 


In this case, the algorithm will use a built-in function:
gatesetbasis_constrction_from_basis_gates() from 'qiskit_experiment/library/gateset_tomography/gatesetbasis.py'

This function, takes basis gates, and construct spam gates which are informationally complete.

For example, the provided gate set can simply be:

In [7]:
# The basis gates are provided as a dictionary similar to the one we pass to the GateSetBasis class.
gateset_3 = {
 'Id': lambda circ, qubit: None,
 'H': lambda circ, qubit: circ.append(HGate(), [qubit]),
 'Y': lambda circ, qubit: circ.append(YGate(), [qubit]),
 'X_Rot_90': lambda circ, qubit: circ.append(U2Gate(-np.pi / 2, np.pi / 2), [qubit]),
}

only_basis_gates_3 = True

## Running gate set tomography on 1-qubit

In [8]:

#First option: Arbitrary gate set provided by the user as a GateSetBasis instance:
#gstexp1 = GateSetTomography(qubits=[0], gateset=gateset_1, additional_gates=[HGate()], 
#                            only_basis_gates=False)

#Second option: Default gate set:
gstexp1 = GateSetTomography(qubits=[0], gateset=gateset_2, additional_gates=[HGate()], 
                            only_basis_gates=False)

#Third option:Providing only basis gates without SPAM:
#gstexp1 = GateSetTomography(qubits=[0], gateset=gateset_3,additional_gates=[HGate()] 
#                            only_basis_gates=True)

#Run GST experiment
gstdata1 = gstexp1.run(backend).block_for_results()

#View final analysis results
for result in gstdata1.analysis_results():
    print(result)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 2.22953462600708, 'fitter_initial_guess': 'linear_inversion', 'GST gates': ['Id', 'SX', 'RZ_pi/2', 'RZ_-pi/2', 'h'], 'GST SPAM gates': {'F0': ('Id',), 'F1': ('SX',), 'F2': ('RZ_pi/2', 'SX', 'RZ_-pi/2'), 'F3': ('SX', 'SX')}}
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of E
- value: Operator([[0.98887364+0.j        , 0.00170124-0.00170124j],
          [0.00170124+0.00170124j, 0.00925987+0.j        ]],
         input_dims=(2,), output_dims=(2,))
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of rho
- value: DensityMatrix([[9.99999792e-01+0.00000000e+00j,
                6.57398415e-08-6.57398415e-08j],
               [6.57398415e-08+6.57398415e-08j,
                2.40143595e-07+0.00000000e+00j]],
              dims=(2,))
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- nam

### Tomography Results

The main results of GST experiment, are the fitted native state and measurement and all the gates in the gate set.

In [9]:
native_state_result = gstdata1.analysis_results("gst estimation of rho")
print(native_state_result.value)

DensityMatrix([[9.99999792e-01+0.00000000e+00j,
                6.57398415e-08-6.57398415e-08j],
               [6.57398415e-08+6.57398415e-08j,
                2.40143595e-07+0.00000000e+00j]],
              dims=(2,))


In [10]:
native_measurement_result = gstdata1.analysis_results("gst estimation of E")
print(native_measurement_result.value)

Operator([[0.98887364+0.j        , 0.00170124-0.00170124j],
          [0.00170124+0.00170124j, 0.00925987+0.j        ]],
         input_dims=(2,), output_dims=(2,))


In [40]:
SX_gate_result = gstdata1.analysis_results("gst estimation of SX")
print(SX_gate_result.value)

Choi([[0.5190236 +0.j        , 0.13568902+0.45431291j,
       0.15719955+0.47222981j, 0.48020383-0.09953489j],
      [0.13568902-0.45431291j, 0.51352484+0.j        ,
       0.45598422-0.01211562j, 0.0012199 -0.46973837j],
      [0.15719955-0.47222981j, 0.45598422+0.01211562j,
       0.47927342+0.j        , 0.05341016-0.46778436j],
      [0.48020383+0.09953489j, 0.0012199 +0.46973837j,
       0.05341016+0.46778436j, 0.48817814+0.j        ]],
     input_dims=(2,), output_dims=(2,))


### Additional state metadata
Additional data about the fidelity is stored in the results extra.
The fidelity measure, computes the avarage gate fidelity of GST results and some target using the function: 'qiskit.quantum_info.average_gate_fidelity'. If the target and the GST result are both non unitary (which is usually the case when the target is noisy), the qiskit.quantum_info.process_fidelity will be used.

The target's default, as in the above experiment, is the ideal gates. However, the user can provide 
an arbitrary target as in the next example.

To show the fidelity of the gates with the default target of the previous example, we type:

In [12]:
print('extra:\n', gstdata1.analysis_results("gst estimation of SX").extra)

extra:
 {'Average gate fidelity': 0.9667089171844322}


In [14]:
print('extra:\n', gstdata1.analysis_results("gst estimation of RZ_pi/2").extra)

extra:
 {'Average gate fidelity': 0.9866344296912347}


### Comparing the GST results to an arbitrary target set

The user can provide an arbitrary target set to compare the results to.
The target set is a dictionary, where the keys are the names of the gates (including E and rho),
and the corresponding values provided as Operator, Gate, FunctionType, PTM or Choi. 
target_set is passed in as an analysis option.

For example:

In [41]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit import Aer

gateset_basis = gateset_3 

#Noise model: Amplitude damping applied on each qubit after each gate. Th PTM representation of
#amplitude damping channel: 
gamma = 0.05
noise_ptm_AD = PTM(np.array([[1, 0, 0, 0],
                             [0, np.sqrt(1 - gamma), 0, 0],
                             [0, 0, np.sqrt(1 - gamma), 0],
                             [gamma, 0, 0, 1 - gamma]]))
         
# We can add targets E and rho, but they are not relevant, as the fidelity measure is applied only 
#on the target set.
target_set = {
 'Id':Choi(IGate()),
 'H': Choi(HGate()),
 'Y': Choi(YGate()),
 'X_Rot_90': Choi(U2Gate(-np.pi / 2, np.pi / 2)),
}
         
#As the gates after the noise in PTM representation is simply the multiplication of the noise
#channel by each gate channel, the noisy target set can be obtained in the PTM representation via:
target_set_noisy = {}
for key in target_set:
    if key is not 'I':
        target_set_noisy[key]= PTM(np.dot(noise_ptm_AD, PTM(target_set[key]).data))
    
#Noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_ptm_AD, ['sx', 'x', 'rz'])
backend_qasm = Aer.get_backend('qasm_simulator')

#GST experiment:
gstexp4=GateSetTomography(qubits=[0], gateset=gateset_3, only_basis_gates=True)
gstexp4.set_analysis_options(target_set = target_set_noisy)

gstdata4 = gstexp4.run(backend=backend_qasm, noise_model=noise_model).block_for_results()

additional_gate_result = gstdata4.analysis_results("gst estimation of H")
print('GST estimation of H:\n', additional_gate_result.value)
print('extra:\n', additional_gate_result.extra)



<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-41-792bbddba1ff>:27: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not 'I':
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.01528154  0.01528154]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.01462444  0.01462444]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.02094982  0.02094982]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.17694236  0.17694236]


GST estimation of H:
 Choi([[ 0.50011885+0.j        ,  0.47802155-0.03416989j,
        0.4838061 -0.01559969j, -0.49240045+0.02350927j],
      [ 0.47802155+0.03416989j,  0.49970625+0.j        ,
        0.46387217-0.02054228j, -0.48644554+0.00121647j],
      [ 0.4838061 +0.01559969j,  0.46387217+0.02054228j,
        0.50549751+0.j        , -0.48904736-0.00607059j],
      [-0.49240045-0.02350927j, -0.48644554-0.00121647j,
       -0.48904736+0.00607059j,  0.49467739+0.j        ]],
     input_dims=(2,), output_dims=(2,))
extra:
 {'Process fidelity': 0.978508093140756}


### Tomography Fitters and additional analysis options:

There are two fitters that can be used for the GST experiment analysis, which can be set as analysis options, by setting fitter = 'name of the fitter'.

$\bf{1.}$ The first fitter is the $\bf{linear \ inversion \ fitter}$: 'linear_inversion_gst'. The rsults of this fitter as mentioned earlier are not physical. Namely,
they are not TP or CP. The user however can choose to rescale the results to be TP or not CP or both. To do this, the user
should set the analysis option $\bf{rescale}$ _ ${TP}$ and $\bf{rescale}$_${CP}$ to be true. The default value of rescale_TP and rescale_CP is False for both. The values of both will be displayed in
the 'GST Experiment properties' AnalysisResult.

$\bf{2.}$ The second fitter which is the default fitter is the $\bf{MLE \ fitter}$: 'scipy_optimizer_MLE_gst'. The results of this fitter are always CPTP and hence, rescale_TP and rescale_CP options are not relevant here. 
For this fitter, a fitter_initial_guess can be passed in as an analysis option. If the fitter_initial_guess is None, the optimization will be performed without an initial point and it may
fail. It also takes as a string either "default" or "linear_inversion" and in both cases it uses linear inversion result as a starting solution. 

The user can provide an arbitrary initial guess for the MLE fitter which is a dictionary, including the names of the state, measurements and gates and their corresponding values, where:

- The value of rho can be a DensityMatrix or an array representing the PTM representation in the orthonormal
Pauli basis composed of strings of the form: $$\frac{1}{2^{N/2}}\Pi_{i_0 i_1 \cdots i_N} P_{i_0}\otimes P_{i_1}\otimes \cdots \otimes P_{i_{N-1}}$$, where $P_{i_j}\in [I, X, Y, Z]$ and 
$N$ is the number of qubits. 
- The value of E is an array representing the PTM representation of E. 
- The value of each one of the gates can be of type: FunctionType, Choi, PTM, Gate or Operator.

$\bf{Note:}$ In the examples above, the default fitter which is the MLE fitter, with the default fitter 
initial guess which is the linear inversion results, were used.

#### Example for displaying the rescaled linear inversion GST results:

In [42]:
gstexp5 = GateSetTomography(qubits=[0],gateset=gateset_1)
gstexp5.set_analysis_options(fitter='linear_inversion_gst',rescale_TP=True, rescale_CP=True)
gstdata5 = gstexp5.run(backend).block_for_results()
print(gstdata5.analysis_results("GST Experiment properties"))
print(gstdata5.analysis_results("gst estimation of X_Rot_90"))
print('extra:\n', gstdata5.analysis_results("gst estimation of X_Rot_90").extra)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'linear_inversion_gst', 'fitter_time': 0.8010118007659912, 'rescale_CP': True, 'rescale_TP': True, 'GST gates': ['Id', 'X_Rot_90', 'Y_Rot_90'], 'GST SPAM gates': {'F0': ('Id',), 'F1': ('X_Rot_90',), 'F2': ('Y_Rot_90',), 'F3': ('X_Rot_90', 'X_Rot_90')}}
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of X_Rot_90
- value: Choi([[ 0.52021125+0.j        ,  0.00590678+0.48343384j,
       -0.00598464+0.49323409j,  0.47633078-0.00414032j],
      [ 0.00590678-0.48343384j,  0.4841892 +0.j        ,
        0.47852852-0.02417098j, -0.01530775-0.47812161j],
      [-0.00598464-0.49323409j,  0.47852852+0.02417098j,
        0.51527237+0.j        ,  0.01669942-0.48923003j],
      [ 0.47633078+0.00414032j, -0.01530775+0.47812161j,
        0.01669942+0.48923003j,  0.48032718+0.j        ]],
     input_dims=(2,), output_dims=(2,))
- extra: <1 items>
- device_components: ['Q0']
- verified: False

#### Example for providing an arbitrary fitter initial guess for MLE fitter:

In [43]:
fitter_initial_gateset = {
 'E': np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]),
 'rho': DensityMatrix(np.array([[1,0],[0,0]])),
 'Id':PTM(IGate()),
 'H': PTM(HGate()),
 'Y': PTM(YGate()),
 'X_Rot_90': PTM(U2Gate(-np.pi / 2, np.pi / 2)),
}


gstexp6 = GateSetTomography(qubits=[0],gateset=gateset_3, only_basis_gates=True)
gstexp6.set_analysis_options(fitter_initial_guess=fitter_initial_gateset, rescale_TP=True, rescale_CP=True)
gstdata6 = gstexp6.run(backend).block_for_results()
print(gstdata6.analysis_results("GST Experiment properties"))
print(gstdata6.analysis_results("gst estimation of H"))
print('extra:\n', gstdata6.analysis_results("gst estimation of H").extra)



DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 2.910433053970337, 'fitter_initial_guess': {'E': array([0.70710678, 0.        , 0.        , 0.70710678]), 'rho': array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j]]), 'Id': PTM([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
     [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
     [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
     [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'H': PTM([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
     [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
     [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'Y': PTM([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'X_Rot_90': PTM([[ 1.000000e+00

## Running gate set tomography on 2-qubits

GST experiment can be run on an arbitrary number of qubits. However, as solving the highly nonlinear MLE optimization problem takes a long time which sharply increases as we increase the number of qubits, it is almost non possible to run GST on three qubits with MLE fitter.

To run GST on two qubits:

In [21]:
qstexp7 = GateSetTomography(qubits=[0,1], gateset='default')
qstexp7.set_analysis_options(fitter='scipy_optimizer_MLE_gst', fitter_initial_guess='default')
qstdata7 = qstexp7.run(backend).block_for_results()
print(qstdata7.analysis_results("GST Experiment properties"))
print(qstdata7.analysis_results("gst estimation of CX"))
print('extra:\n', qstdata7.analysis_results("gst estimation of CX").extra)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 1905.7633047103882, 'fitter_initial_guess': 'linear_inversion', 'GST gates': ['I I', 'X I', 'I X', 'RZ_pi_over_3 I', 'I RZ_pi_over_3', 'I SX', 'SX I', 'CX'], 'GST SPAM gates': {'F0': ('I I',), 'F1': ('X I',), 'F2': ('I X',), 'F3': ('X I', 'I X'), 'F4': ('I X', 'SX I', 'CX'), 'F5': ('I SX', 'I RZ_pi_over_3', 'I SX'), 'F6': ('SX I', 'RZ_pi_over_3 I', 'SX I'), 'F7': ('X I', 'I SX', 'I RZ_pi_over_3', 'I SX'), 'F8': ('I X', 'I SX', 'CX', 'I SX'), 'F9': ('I X', 'SX I', 'RZ_pi_over_3 I', 'SX I'), 'F10': ('RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'SX I'), 'F11': ('I RZ_pi_over_3', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I SX'), 'F12': ('I SX', 'SX I', 'CX', 'I SX'), 'F13': ('X I', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I SX'), 'F14': ('I SX', 'I RZ_pi_over_3', 'SX I', 'CX', 'I SX'), 'F15': ('RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'CX', 'I SX'

## References

[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." arXiv preprint arXiv:1509.02921 (2015).

In [45]:
import qiskit.tools.jupyter
%qiskit_copyright